## NER-SYM(Named Entity recognition using symbolic NLP)

                                                                        (By :- Sai Hemanth Kilaru)

Note: I have tried my best and I have completed the project. i have referred to multiple web sources like stack overflow ,etc.. Here are the few links of the sources i have referred to:
1. for regex:  'https://docs.python.org/3/library/re.html'
2. for nltk and spacy: 'https://towardsdatascience.com/named-entity-recognition-with-nltk-and-spacy-8c4a7d88e7da
'
3. for evaluation metrics: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html   and my ML notebook.

4. Assistant professor Kadir Bulut- helped me in solving few errors in the  code part.


#### Importing the libraries

In [1]:
import nltk
import spacy
import re

#we download the punkt and average perceptron trigger - used for tokenization and part of speech tagging,maxnet_ne_chunker-chunking process
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
# we download SpaCy's English model 
spacy.cli.download("en_core_web_sm")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


#### Loading the dataset


we can get the WikiANN dataset  from the hugging face,directly by loading it from the datasets library.As here are multiple languages in the dataset, we import the one related to english 'en'.
original dataset link: https://huggingface.co/datasets/unimelb-nlp/wikiann

In [2]:
# loading the  WikiANN dataset (English) from the hugging face with the help of datasets library.
from datasets import load_dataset

dataset = load_dataset("wikiann", "en")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
#Just to look at the dataset, i have used pandas here. tbh,there's no use of it here other than this.
import pandas as pd
#converting it into a pandas dataframe
df_train = pd.DataFrame(dataset["train"])
#first 5 rows of the dataset
df_train.head()

,tokens,ner_tags,langs,spans
0,"[R.H., Saunders, (, St., Lawrence, River, ), (...","[3, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0]","[en, en, en, en, en, en, en, en, en, en, en]","[ORG: R.H. Saunders, ORG: St. Lawrence River]"
1,"[;, ', '', Anders, Lindström, '', ']","[0, 0, 0, 1, 2, 0, 0]","[en, en, en, en, en, en, en]",[PER: Anders Lindström]
2,"[Karl, Ove, Knausgård, (, born, 1968, )]","[1, 2, 2, 0, 0, 0, 0]","[en, en, en, en, en, en, en]",[PER: Karl Ove Knausgård]
3,"[Atlantic, City, ,, New, Jersey]","[5, 6, 6, 6, 6]","[en, en, en, en, en]","[LOC: Atlantic City , New Jersey]"
4,"[Her, daughter, from, the, second, marriage, w...","[0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, ...","[en, en, en, en, en, en, en, en, en, en, en, e...","[PER: Marie d'Agoult, PER: Franz Liszt, PER: C..."


In [4]:
#shape
df_train.shape

(20000, 4)

In [5]:
#checking the stats
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tokens    20000 non-null  object
 1   ner_tags  20000 non-null  object
 2   langs     20000 non-null  object
 3   spans     20000 non-null  object
dtypes: object(4)
memory usage: 625.1+ KB


In [6]:
#load the spacy's english model which we have downloaded earlier
nlp = spacy.load("en_core_web_sm")

#### Data Preprocessing
Our Symbolic NER system focusses on extracting  the named entities from the WikiANN dataset.So,we should avoid applying stemming or lemmatization such that it preserve the integrity and enhances the accuracy of NER results.(Based on my research(I have tried to use it on the earlier code chunks,I found the current approach better))

In [7]:

from nltk import pos_tag, ne_chunk
from spacy import displacy
#We create a preprocess function which will tokenize the text, apply POS tagging, and use chunking(for multi-word entities)
def preprocess(text):
    tokens = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    chunks = nltk.ne_chunk(pos_tags)
    return tokens, pos_tags, chunks


##### Entity Extraction functions

Here,the entities are: Names, Dates, Locations, and Organizations. we use different functions witht the help of chunking and regex to extract these from the data.

For the regex part down there , i have learnt regular expressions last year through https://www.youtube.com/watch?v=vsa9GGzMFXQ . This lecture is intermediate-level and contained the few necessary topics i need.i revised it once again before doing this ; which helped me with the entity extraction process in writing the few functions. 

In [8]:
#function to identify the person names(chunking is used). I have added appending the remaining name part; in order to retreive the full name
def extract_names(chunks):
    names = []
    current_name = []

    for chunk in chunks:
        # Check if the chunk is a named entity of type 'PERSON'
        if hasattr(chunk, 'label') and chunk.label() == 'PERSON':
            current_name.append(" ".join(c[0] for c in chunk))
        else:
            if current_name:
                # Append the  name accumulated
                names.append(" ".join(current_name))
                current_name = []  # reset- for the next potential name

    if current_name:  # append other name, it goes on....
        names.append(" ".join(current_name))

    return names


In [9]:
#testing my function with 2 sample strings
# consider two sample strings(taken from the web and added few more words to it )
sample_text_1 = "Marie Curie was born in 19 May 1867 and worked in Paris in  university of az."
sample_text_2 = "Albert Einstein developed the theory of relativity in Germany, 1947 in office."

# Testing Name Extraction, we already have a preprocessing function above, just for the checking sake , i have done all the operations again
tokens_1 = nltk.word_tokenize(sample_text_1)
chunks_1 = ne_chunk(nltk.pos_tag(tokens_1))
print(f"Text: {sample_text_1}")
print(f"Extracted Names: {extract_names(chunks_1)}\n")

tokens_2 = nltk.word_tokenize(sample_text_2)
chunks_2 = ne_chunk(nltk.pos_tag(tokens_2))
print(f"Text: {sample_text_2}")
print(f"Extracted Names: {extract_names(chunks_2)}\n")


Text: Marie Curie was born in 19 May 1867 and worked in Paris in  university of az.
Extracted Names: ['Marie Curie']

Text: Albert Einstein developed the theory of relativity in Germany, 1947 in office.
Extracted Names: ['Albert Einstein']



In [10]:
#function to identify the dates.(with the help of regex date pattern) [Took help from the professor 'bulut'] and inspired from the above source. 
#Here, at starting , i have thought of writing a regex code only in the MM-DD-YYYY format . but i  have observed that most of the texts consists of year , date and month seperately, so i have thought of going for the seperate functions.

import re
def extract_dates(text):
    # extracting the years
    year_pattern = r'\b(?:18|19|20)\d{2}\b'  # matches the years
    years = re.findall(year_pattern, text)

    # extracting months in both the formats(example:Jan, January)
    month_pattern = r'\b(January|February|March|April|May|June|July|August|September|October|November|December|' \
                    r'Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\b'
    months = re.findall(month_pattern, text)

    # extracting the  day/s
    day_pattern = r'\b([1-9]|[12][0-9]|3[01])(st|nd|rd|th)?\b'
    days = re.findall(day_pattern, text)
    days = [day[0] for day in days]  # Extract the day numbers from the tuple

    # we ll return components that are found in the sentence,if not then none
    date_components = {
        'Years': years if years else None,
        'Months': months if months else None,
        'Days': days if days else None
    }

    # Filtering  out the None values and return
    return {k: v for k, v in date_components.items() if v}


In [11]:
# Testing the extraction date function
print(f"Text: {sample_text_1}")
print(f"Extracted Dates: {extract_dates(sample_text_1)}\n")

print(f"Text: {sample_text_2}")
print(f"Extracted Dates: {extract_dates(sample_text_2)}\n")


Text: Marie Curie was born in 19 May 1867 and worked in Paris in  university of az.
Extracted Dates: {'Years': ['1867'], 'Months': ['May'], 'Days': ['19']}

Text: Albert Einstein developed the theory of relativity in Germany, 1947 in office.
Extracted Dates: {'Years': ['1947']}



In [12]:
#function to identify the locations(chunking is used ,we check proper nouns/prepositions indicating the locations)(similar style to name function)
def extract_locations(chunks, tagged_tokens):
    locations = []
    temp_location = []

    for chunk in chunks:
        # Check if the chunk is a named entity of type GPE (Geopolitical Entity)
        if hasattr(chunk, 'label') and chunk.label() == 'GPE':
            temp_location.append(" ".join(c[0] for c in chunk))
        elif temp_location:
            # append that location
            locations.append(" ".join(temp_location))
            temp_location = []  # reset- for the next potential location

    if temp_location:  # append  remaining locations
        locations.append(" ".join(temp_location))

    return locations




In [13]:
#testing extract location function for the above sample text
print(f"Text: {sample_text_1}")
print(f"Extracted Locations: {extract_locations(chunks_1, nltk.pos_tag(tokens_1))}\n")

print(f"Text: {sample_text_2}")
print(f"Extracted Locations: {extract_locations(chunks_2, nltk.pos_tag(tokens_2))}\n")


Text: Marie Curie was born in 19 May 1867 and worked in Paris in  university of az.
Extracted Locations: ['Paris']

Text: Albert Einstein developed the theory of relativity in Germany, 1947 in office.
Extracted Locations: ['Germany']



In [14]:
#function to identify the organization.(with the help of spacy's named entity recognition(as per my research,this is the simplier way to do it(without that, it gave a very less accuracy earlier))(source: already mentioned above )

def extract_organizations(text):
    doc = nlp(text)  # Processing the text with SpaCy
    organizations = [ent.text for ent in doc.ents if ent.label_ == "ORG"]  # Extraction of organizations
    return organizations



In [15]:
# Sample strings to check this(took from web and modified as it contains te org name)
sample_text_3 = "Marie Curie worked with University of Paris."
sample_text_4 = "He is a scientist at Google."

# Testing Organization Extraction
print(f"Text: {sample_text_3}")
print(f"Extracted Organizations: {extract_organizations(sample_text_3)}\n")

print(f"Text: {sample_text_4}")
print(f"Extracted Organizations: {extract_organizations(sample_text_4)}\n")


Text: Marie Curie worked with University of Paris.
Extracted Organizations: ['University of Paris']

Text: He is a scientist at Google.
Extracted Organizations: ['Google']



##### Combining all these individual entities into Symbolic NER System

In [16]:
def symbolic_ner(text):
    tokens, pos_tags, chunks = preprocess(text)  # to preprocess text
    names = extract_names(chunks)  # to extract names
    dates = extract_dates(text)  # to extract dates
    locations = extract_locations(chunks, pos_tags)  # to extract locations
    organizations = extract_organizations(text)  # to extract organizations

    # we ll create a dictionary where the keys are entity types and the values are extracted entities from the text
    entities = {
        'Names': names,
        'Dates': dates,
        'Locations': locations,
        'Organizations': organizations
    }

    return entities

##### Model Evaluation

In [17]:
# now,we ll write a function to test the system on the entire dataset and then call it
def test_full_dataset(dataset):
  #intialise the following:
  #For the better model evaluation, i have decided to create a counter seperately for the correct and total entities
    symbolic_correct_names = 0
    symbolic_total_names = 0
    symbolic_correct_dates = 0
    symbolic_total_dates = 0
    symbolic_correct_locations = 0
    symbolic_total_locations = 0
    symbolic_correct_organizations = 0
    symbolic_total_organizations = 0
  #create a empty list to store the detailed observations
    detailed_observations = []

    # iterate through all samples in the dataset
    for sample in dataset["train"]:  # through the entire dataset
        sentence = " ".join(sample["tokens"])  # joining the  tokens to form a complete sentence
        entities = symbolic_ner(sentence)  # getting the  entities

        # extracting the true entities
        true_names = extract_names(ne_chunk(nltk.pos_tag(sample["tokens"])))# names
        true_dates = extract_dates(sentence)# dates
        true_locations = extract_locations(ne_chunk(nltk.pos_tag(sample["tokens"])), nltk.pos_tag(sample["tokens"]))#locations
        true_organizations = extract_organizations(sentence)#organisations
#now we ll update the total entities and ll take the count of the correct entities.
        # evaluating the names(This whole process till the overall accuracy,the process has been inspired from a few comments on the stack overflow and also got a little help from the professor 'bulut')
        symbolic_total_names += len(true_names)
        symbolic_correct_names += len(set(true_names) & set(entities['Names']))

        # evaluating the dates
        symbolic_total_dates += len(true_dates)
        symbolic_correct_dates += len(set(true_dates) & set(entities['Dates']))

        # evaluating the locations
        symbolic_total_locations += len(true_locations)
        symbolic_correct_locations += len(set(true_locations) & set(entities['Locations']))

        # evaluating the  organizations
        symbolic_total_organizations += len(true_organizations)
        symbolic_correct_organizations += len(set(true_organizations) & set(entities['Organizations']))

        # we ll create detailed observation for this sample (for the better understanding)
        observation = {
            'Sentence': sentence,
            'Entities Found': entities,
            'True Names': true_names,
            'Correctly Identified Names': set(true_names) & set(entities['Names']),
            'True Dates': true_dates,
            'Correctly Identified Dates': set(true_dates) & set(entities['Dates']),
            'True Locations': true_locations,
            'Correctly Identified Locations': set(true_locations) & set(entities['Locations']),
            'True Organizations': true_organizations,
            'Correctly Identified Organizations': set(true_organizations) & set(entities['Organizations'])
        }
        detailed_observations.append(observation)

    # calculate overall accuracy for all entities, we divie these two , such that we get the exact accuracy for our entities()
    symbolic_accuracy_names = symbolic_correct_names / symbolic_total_names if symbolic_total_names > 0 else 0
    symbolic_accuracy_dates = symbolic_correct_dates / symbolic_total_dates if symbolic_total_dates > 0 else 0
    symbolic_accuracy_locations = symbolic_correct_locations / symbolic_total_locations if symbolic_total_locations > 0 else 0
    symbolic_accuracy_organizations = symbolic_correct_organizations / symbolic_total_organizations if symbolic_total_organizations > 0 else 0

    # we ll print  overall accuracy for each entity type(for better understanding)
    print(f"\nOverall Accuracy of Symbolic NER Model for Names: {symbolic_accuracy_names:.4f}")
    print(f"Overall Accuracy of Symbolic NER Model for Dates: {symbolic_accuracy_dates:.4f}")
    print(f"Overall Accuracy of Symbolic NER Model for Locations: {symbolic_accuracy_locations:.4f}")
    print(f"Overall Accuracy of Symbolic NER Model for Organizations: {symbolic_accuracy_organizations:.4f}")




In [18]:
# Call the function to test the full dataset
test_full_dataset(dataset)



Overall Accuracy of Symbolic NER Model for Names: 0.9793
Overall Accuracy of Symbolic NER Model for Dates: 1.0000
Overall Accuracy of Symbolic NER Model for Locations: 0.9828
Overall Accuracy of Symbolic NER Model for Organizations: 0.9984


From the above code observations , we came to know that the symbolic nlp for the NER works well for the date,organization followed by the name and location(in very few cases among the 20000 samples). This name function's accuracy can be improved by adding the PERSON label of spacy's a;ong with the names function and GPE and LOC label of spacy's for the location function , this ll result in a higher accuracy. But, as my project is related to symbolic nlp specifically, i have erased those added codelines , as it seemed to reduce the symbolic nlp's significance in my Project. The date function worked well when we considered extracting the day,month,year instead of the DD/MM/YYYY format.

ran the code in : google collab

## THANK YOU!